# Auto-encodeurs débruiteurs avec Keras

## Vérification de l'utilisation de GPU

Allez dans le menu `Exécution > Modifier le type d'execution` et vérifiez que l'on est bien en Python 3 et que l'accélérateur matériel est configuré sur « GPU ».

In [ ]:
!nvidia-smi

## Import de TensorFlow et des autres librairies nécessaires

In [ ]:
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Chargement de MNIST

Nous allons utiliser un prétraîtement légèrement différent des autres fois : étant donné que nous voulons pouvoir prédire les valeurs données en entrée en sortie (principe de l'auto-encodage), nous allons simplement projeter ces valeurs dans $[0, 1]$ au lieu de $[0, 255]$. Notez qu'habituellement nous ne faisons pas ça : nous normalisons en centrant sur zéro et en divisant par l'écart-type.

In [ ]:
(X_train, _), (X_test, y_test) = keras.datasets.mnist.load_data()
nb_classes = 10
input_dim = 28 * 28
X_train = X_train.reshape(-1, input_dim).astype('float32')
X_test = X_test.reshape(-1, input_dim).astype('float32')

# On utilise cette normalisation pour garder les pixel à 0 où ils sont
X_train = X_train / 255.0
X_test = X_test / 255.0

## Application d'un bruit  gaussien

In [ ]:
noise_factor = 0.3
X_train_noisy = X_train + numpy.random.normal(0, noise_factor, X_train.shape) 
X_test_noisy = X_test + numpy.random.normal(0, noise_factor, X_test.shape)

# On clip les valeurs pour éviter les pixels plus blanc que blanc (ou plus noir
# que noir) 
numpy.clip(X_train_noisy, 0, 1, out=X_train_noisy)
numpy.clip(X_test_noisy, 0, 1, out=X_test_noisy)

In [ ]:
n = 10
f, ax = plt.subplots(1, n, figsize=(n * 1.4, 2))
for i in range(n):
    ax[i].imshow(X_test_noisy[i].reshape(28, 28), cmap="gray_r")
    ax[i].set_title(y_test[i])
    ax[i].axis("off")
plt.show()

## Création de l'autoencodeur débruiteur



In [ ]:
# Votre code ici
encoding_dim = 4

autoencoder = keras.Sequential()

## Apprentissage

*Écrivez la ligne correspondant à l'apprentissage de votre autoencodeur :*

- *50 itérations devraient suffire*
- *Utilisez un batch de 256*

In [ ]:
# Votre code ici

## Base de Test

Autoencodez les images de test et stockez les images obtenues dans la variable `X_test_noisy_pred`

In [ ]:
# Votre code ici
X_test_noisy_pred = X_test_noisy

## Affichage visuel de la performance

In [ ]:
n = 10
_, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))
for i, (ax_top, ax_bottom) in enumerate(ax.T):
    # L'original en haut
    ax_top.imshow(X_test_noisy[i].reshape(28, 28), cmap="gray_r")
    ax_top.set_title(str(y_test[i]))
    ax_top.axis("off")

    # La reconstruction en bas
    ax_bottom.imshow(X_test_noisy_pred[i].reshape(28, 28), cmap="gray_r")
    ax_bottom.axis("off")
plt.show()

## Essayez avec plus de neurones

Que se passe-t-il ?

## Utilisation des réseaux convolutifs

Pour cela il faut remettre chaque image sous forme 28x28x1. Les CNNs ont besoin de cette 3ème dimension de tenseur (il pourrait y avoir plus de canaux que le niveau de gris : il y en a 3 pour les images en couleur et plus encore dans les couches intermédiaires d'un réseau convolutif où le nombre de canaux en entrée d'une couche sera le nombre de kernels de la couche précédente).

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train_noisy = X_train_noisy.reshape(-1, 28, 28, 1)
X_test_noisy = X_test_noisy.reshape(-1, 28, 28, 1)

## Création du modèle

On utilisera des séquences de [`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/), [`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/) avec des kernel de respectivement 3 × 3 et 2 × 2 pour la partie encodeur. Dans ces deux layers, il faudra utiliser l'option `padding="same"` afin d'éviter les effets de bord (l'image étant déjà assez petite comme ça).

Pour la partie décodeur, on utilisera des [`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/) de même nature suivis par des [`UpSampling2D`](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) (de taille 2 × 2) qui correspondent à l'opération inverse de [`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/).

N'hésitez pas à abuser de `model.summary()` pour vous y retrouver. L'objectif étant de retrouver une image 28 × 28 × 1 à la sortie du dernier [`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/). En effet, finir par un [`UpSampling2D`](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) serait une mauvaise idée.

Le réseau va être profond, on utilisera des fonctions d'activation ReLU, sauf pour la dernière où on utilisera une sigmoïde.

In [ ]:
# Votre code ici
autoencoder = keras.Sequential()

## Apprentissage

*Écrivez la ligne correspondant à l'apprentissage de votre autoencodeur :*

- *100 itérations devraient suffire*
- *Utilisez un batch de 256*

In [ ]:
# Votre code ici

## Affichage des performances

In [ ]:
X_test_noisy_pred = autoencoder.predict(X_test_noisy).reshape(-1, 28, 28)

n = 10

random_indexes = numpy.random.choice(X_test.shape[0],
                                     replace=False,
                                     size=n)

_, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))
for (ax_top, ax_bottom), random_index in zip(ax.T, random_indexes):
    # L'image originale en haut
    ax_top.set_title(str(y_test[random_index]))
    ax_top.imshow(X_test_noisy[random_index].reshape(28, 28), cmap="gray_r")
    ax_top.axis("off")

    # L'image reconstruite en bas
    ax_bottom.imshow(X_test_noisy_pred[random_index], cmap="gray_r")
    ax_bottom.axis("off")
plt.show()

## Sur des images non-bruitées

In [ ]:
X_test_pred = autoencoder.predict(X_test).reshape(-1, 28, 28)

n = 10

random_indexes = numpy.random.choice(X_test.shape[0],
                                     replace=False,
                                     size=n)

_, ax = plt.subplots(2, n, figsize=(n * 1.4, 4))
for (ax_top, ax_bottom), random_index in zip(ax.T, random_indexes):
    # L'image originale en haut
    ax_top.set_title(str(y_test[random_index]))
    ax_top.imshow(X_test[random_index].reshape(28, 28), cmap="gray_r")
    ax_top.axis("off")

    # L'image reconstruite en bas
    ax_bottom.imshow(X_test_pred[random_index], cmap="gray_r")
    ax_bottom.axis("off")
plt.show()